# Sample for KFServing SDK with a custom image

This is a sample for KFServing SDK using a custom image.

The notebook shows how to use KFServing SDK to create, get and delete InferenceService with a custom image.

### Setup
- Your `~/.kube/config` should point to a cluster with KFServing installed.
- Your cluster's Istio Ingress gateway must be network accessible.

### Build the docker image we will be using.

The goal of custom image support is to allow users to bring their own wrapped model inside a container and serve it with KFServing. Please note that you will need to ensure that your container is also running a web server e.g. Flask to expose your model endpoints. This example extends kfserving.KFModel which uses the tornado web server.


To build and push with Docker Hub set the `DOCKER_HUB_USERNAME` variable below with your Docker Hub username

In [ ]:
### Github: https://github.com/kserve/kserve/tree/v0.6.0/docs/samples/v1alpha2/custom/kfserving-custom-model

In [ ]:
# Set this to be your dockerhub username
# It will be used when building your image and when creating the InferenceService for your image
DOCKER_HUB_USERNAME = "your_docker_username"

In [ ]:
%%bash -s "$DOCKER_HUB_USERNAME"
docker build -t $1/kfserving-custom-model ./model-server

In [ ]:
%%bash -s "$DOCKER_HUB_USERNAME"
docker push $1/kfserving-custom-model

### KFServing Client SDK

We will use the [KFServing client SDK](https://github.com/kubeflow/kfserving/blob/master/python/kfserving/README.md#kfserving-client) to create the InferenceService and deploy our custom image.

In [ ]:
from kubernetes import client
from kubernetes.client import V1Container

from kfserving import KFServingClient
from kfserving import constants
from kfserving import utils
from kfserving import V1alpha2EndpointSpec
from kfserving import V1alpha2PredictorSpec
from kfserving import V1alpha2InferenceServiceSpec
from kfserving import V1alpha2InferenceService
from kfserving import V1alpha2CustomSpec

In [ ]:
namespace = utils.get_default_target_namespace()
print(namespace)

### Define InferenceService

Firstly define default endpoint spec, and then define the inferenceservice using the endpoint spec.

To use a custom image we need to use V1alphaCustomSpec which takes a [V1Container](https://github.com/kubernetes-client/python/blob/master/kubernetes/docs/V1Container.md)
 from the kuberenetes library


In [ ]:
api_version = constants.KFSERVING_GROUP + '/' + constants.KFSERVING_VERSION

default_endpoint_spec = V1alpha2EndpointSpec(
                          predictor=V1alpha2PredictorSpec(
                              custom=V1alpha2CustomSpec(
                                  container=V1Container(
                                      name="kfserving-custom-model",
                                      image=f"{DOCKER_HUB_USERNAME}/kfserving-custom-model"))))

isvc = V1alpha2InferenceService(api_version=api_version,
                          kind=constants.KFSERVING_KIND,
                          metadata=client.V1ObjectMeta(
                              name='kfserving-custom-model', namespace=namespace),
                          spec=V1alpha2InferenceServiceSpec(default=default_endpoint_spec))

### Create the InferenceService

Call KFServingClient to create InferenceService.

In [ ]:
KFServing = KFServingClient()
KFServing.create(isvc)

### Check the InferenceService

In [ ]:
KFServing.get('kfserving-custom-model', namespace=namespace, watch=True, timeout_seconds=120)

### Run a prediction 

In [1]:
MODEL_NAME = "kfserving-custom-model"

In [2]:
%%bash --out CLUSTER_IP
INGRESS_GATEWAY="istio-ingressgateway"
echo "$(kubectl -n istio-system get service $INGRESS_GATEWAY -o jsonpath='{.status.loadBalancer.ingress[0].ip}')"

In [11]:
!kubectl -n istio-system get service istio-ingressgateway -o jsonpath='{.status.loadBalancer.ingress[0].ip}'

In [3]:
%%bash -s "$MODEL_NAME" --out SERVICE_HOSTNAME
echo "$(kubectl get inferenceservice $1 -o jsonpath='{.status.url}' | cut -d "/" -f 3)"

In [15]:
!curl -v http://192.168.49.2:31225/v1/models/kfserving-custom-model:predict -d @./input.json

*   Trying 192.168.49.2:31225...
* TCP_NODELAY set
* Connected to 192.168.49.2 (192.168.49.2) port 31225 (#0)








* Mark bundle as not supporting multiuse

* We are completely uploaded and fine
* Mark bundle as not supporting multiuse






* Connection #0 to host 192.168.49.2 left intact
{"predictions": {"Labrador retriever": 0.41585126519203186, "golden retriever": 0.16591677069664001, "Saluki, gazelle hound": 0.1628689467906952, "whippet": 0.028539132326841354, "Ibizan hound, Ibizan Podenco": 0.02392476797103882}}

In [1]:
import requests
import json

with open('input.json') as json_file:
    data = json.load(json_file)
    url ="http://192.168.49.2:31225/v1/models/kfserving-custom-model:predict"
    result = requests.post(url, data=json.dumps(data))
    print(result.content)

b'{"predictions": {"Labrador retriever": 0.41585126519203186, "golden retriever": 0.16591677069664001, "Saluki, gazelle hound": 0.1628689467906952, "whippet": 0.028539132326841354, "Ibizan hound, Ibizan Podenco": 0.02392476797103882}}'


In [6]:
!kubectl describe inferenceservice kfserving-custom-model

Name:         kfserving-custom-model
Namespace:    default
Labels:       controller-tools.k8s.io=1.0
Annotations:  <none>
API Version:  serving.kubeflow.org/v1beta1
Kind:         InferenceService
Metadata:
  Creation Timestamp:  2022-03-12T08:13:29Z
  Finalizers:
    inferenceservice.finalizers
  Generation:  2
  Managed Fields:
    API Version:  serving.kubeflow.org/v1alpha2
    Fields Type:  FieldsV1
    fieldsV1:
      f:metadata:
        f:annotations:
          .:
          f:kubectl.kubernetes.io/last-applied-configuration:
        f:labels:
          .:
          f:controller-tools.k8s.io:
      f:spec:
        .:
        f:default:
          .:
          f:predictor:
            .:
            f:custom:
              .:
              f:container:
                .:
                f:image:
                f:name:
    Manager:      kubectl-client-side-apply
    Operation:    Update
    Time:         2022-03-12T08:13:27Z
    API Version:  serving.kubeflow.org/v1beta1
    Fields T

In [9]:
!curl -v http://kfserving-custom-model.default.svc.cluster.local/v1/models/kfserving-custom-model:predict

* Could not resolve host: kfserving-custom-model.default.svc.cluster.local
* Closing connection 0
curl: (6) Could not resolve host: kfserving-custom-model.default.svc.cluster.local


### Delete the InferenceService

In [ ]:
KFServing.delete(MODEL_NAME, namespace=namespace)